In [33]:
import os
from dotenv import load_dotenv
import time
import numpy as np

In [34]:
load_dotenv()
no_of_keys = 10
keys = []

for x in range(1,no_of_keys+1):
    keys.append(os.getenv(f'GOOGLE_API_KEY_{x}'))

In [35]:
keys

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [36]:
class LoadBalancer:
    def __init__(self,n):
        self.index = -2
        self.no_of_keys = n
        load_dotenv()
        keys = []
        for x in range(1,self.no_of_keys+1):
            keys.append(os.getenv(f'GOOGLE_API_KEY_{x}'))
        self.keys = keys

    def Round_Robin(self):
        self.index += 1
        return self.keys[(self.index+1)%self.no_of_keys]

In [37]:
l = LoadBalancer(10)

In [38]:
l.Round_Robin()

'a'

In [39]:
d = {k:x for x,k in enumerate(keys)}
d

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9}

In [48]:
class LoadBalancer:
    def __init__(self,n,ct):
        self.index = -1
        self.no_of_keys = n
        load_dotenv()
        self.keys = [os.getenv(f'GOOGLE_API_KEY_{x}') for x in range(1,n+1)]
        self.fail_keys = {}
        self.cooltime = ct
        self.usage_count = {key: 0 for key in self.keys}
    
    def StdDev(self):
            avail_keys = self.keys
            
            use_counts = np.array([self.usage_count[key] for key in avail_keys])
            mean_use = np.mean(use_counts)
            std_dev = np.std(use_counts) + 1e-6

            prob = 1 / (1 + np.abs(use_counts - mean_use) / std_dev)
            prob /= prob.sum()

            sel_key = np.random.choice(avail_keys,p=prob)
            self.usage_count[sel_key] += 1

            return sel_key

    def FailureAware(self):
        for _ in range(self.no_of_keys):
            self.index = (self.index+1) % self.no_of_keys
            key = self.keys[self.index]

            if key in self.fail_keys:
                if time.time() < self.fail_keys[key]:
                    continue
                else:
                    del self.fail_keys[key]
            return key
        
        raise Exception("No keys available... All keys in cooldown...")
    
    def report_fail(self,key):
        self.fail_keys[key] = time.time() + self.cooltime
    
    def make_api_call(self, simulate_failure_rate=0.3):
        """
        Simulates an API call using the load balancer.
        :param simulate_failure_rate: Probability (0-1) of a key failing.
        """
        import random

        key = self.FailureAware()
        print(f"Using API key: {d[key]}")

        if random.random() < simulate_failure_rate:
            print(f"API key {d[key]} failed! Putting it in cooldown.")
            self.report_fail(key)
            return False 
        else:
            print(f"API call successful with key {d[key]}")
            return True  

In [58]:
lb = LoadBalancer(10,60)
for x in range(10):
    print(lb.StdDev())

f
i
j
h
b
b
e
i
h
i
